In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#def load_data():
global smart
global gas
global weather
df =pd.read_csv("/datc/opschaler/smartmeter_data/P01S01W0373.csv",header=0,delimiter=";",parse_dates=['Timestamp', 'gasTimestamp'])

# Split the smartmeter and gasmeter data
smart = df.iloc[:,:7]
gas = df.iloc[:, 7:]

In [3]:
#load_data()

# Change all datetime columns to the same column name
smart = smart.rename(index=str, columns={"Timestamp":"datetime"})
gas = gas.rename(index=str, columns={"gasTimestamp":"datetime"})

#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter', 'T', 'Q']]
#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter']]
smart.head()

# Set datetime as index
smart = smart.set_index(['datetime'])
gas = gas.set_index(['datetime'])

In [4]:
"""
Resample the dataframes to 10s intervals.
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
"""
smart = smart.resample('10s').mean()
gas = gas.resample('H').mean() # Does not look nice, but it works

In [5]:
smart.head()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn
datetime,,,,,,
2017-03-07 18:14:20,3717.472,0.0,3883.382,0.0,376.0,0.0
2017-03-07 18:14:30,3717.473,0.0,3883.382,0.0,381.0,0.0
2017-03-07 18:14:40,3717.474,0.0,3883.382,0.0,378.0,0.0
2017-03-07 18:14:50,3717.475,0.0,3883.382,0.0,376.0,0.0
2017-03-07 18:15:00,3717.476,0.0,3883.382,0.0,380.0,0.0


In [6]:
gas.head()

,gasMeter
datetime,
2017-03-07 18:00:00,1599.792
2017-03-07 19:00:00,1599.986
2017-03-07 20:00:00,1600.179
2017-03-07 21:00:00,1600.367
2017-03-07 22:00:00,1600.513


# Working with electricity dataframe

In [7]:
#First check how many nans are there
smart_nan=smart.isnull().any(axis=1)
print(smart[smart_nan].info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2947 entries, 2017-03-07 18:16:10 to 2017-03-28 23:50:30
Data columns (total 6 columns):
eMeter             0 non-null float64
eMeterReturn       0 non-null float64
eMeterLow          0 non-null float64
eMeterLowReturn    0 non-null float64
ePower             0 non-null float64
ePowerReturn       0 non-null float64
dtypes: float64(6)
memory usage: 161.2 KB
None


In [8]:
smart.describe()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn
count,180567.000000,180567.0,180567.000000,180567.0,180567.000000,180567.0
mean,3772.187364,0.0,3943.679690,0.0,459.004615,0.0
std,32.169904,0.0,36.106887,0.0,452.420914,0.0
min,3717.472000,0.0,3883.382000,0.0,238.000000,0.0
25%,3737.922000,0.0,3918.407500,0.0,319.000000,0.0
50%,3775.712000,0.0,3939.401000,0.0,351.000000,0.0
75%,3801.757500,0.0,3973.161000,0.0,414.000000,0.0
max,3830.551000,0.0,4004.212000,0.0,7765.000000,0.0


In [9]:
print(smart[smart_nan])

                     eMeter  eMeterReturn  eMeterLow  eMeterLowReturn  ePower  \
datetime                                                                        
2017-03-07 18:16:10     NaN           NaN        NaN              NaN     NaN   
2017-03-07 18:17:40     NaN           NaN        NaN              NaN     NaN   
2017-03-07 18:46:00     NaN           NaN        NaN              NaN     NaN   
2017-03-07 18:47:20     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:17:10     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:18:30     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:48:10     NaN           NaN        NaN              NaN     NaN   
2017-03-07 19:49:30     NaN           NaN        NaN              NaN     NaN   
2017-03-07 20:18:00     NaN           NaN        NaN              NaN     NaN   
2017-03-07 20:19:20     NaN           NaN        NaN              NaN     NaN   
2017-03-07 20:49:10     NaN 

In [10]:
#Then just interpolate
#smart_final=smart.interpolate('time')
#print(smart_final)

In [11]:
#make ints from NaNs
smart_nan=smart_nan*1
#make a df where the number of consecutive NaNs in that electricity file are displayed
temp = pd.DataFrame()
temp['temp']=smart_nan.groupby((smart_nan != smart_nan.shift()).cumsum()).transform('size') * smart_nan
print(temp.head())

                     temp
datetime                 
2017-03-07 18:14:20     0
2017-03-07 18:14:30     0
2017-03-07 18:14:40     0
2017-03-07 18:14:50     0
2017-03-07 18:15:00     0


In [12]:
temp.to_csv('~/temp.csv')

In [13]:
temp.describe()

,temp
count,183514.000000
mean,1.424965
std,22.504673
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,360.000000


# Creating DF for information from the different files

In [35]:
#create dataframe to store all information from the different files
info_data = pd.DataFrame()
info_data = pd.DataFrame(columns={'House_ID','File_size(MB)','Initial_Date','Final_Date','Days','Amount_of_NaNs','Max_Consecutive_NaNs'})
#order of column names changed for some reason, line(below) fixes this, although ugly
info_data = info_data[['House_ID','File_size(MB)','Initial_Date','Final_Date','Days','Amount_of_NaNs','Max_Consecutive_NaNs']]
info_data.loc[len(info_data.index)] = [123123, smart.memory_usage(index=True).sum()/1000000, smart.index[0], smart.index[-1], smart.index[-1]-smart.index[0], 6767, temp['temp'].max()]
print(info_data.head())

#Where the following things must be changed in order to make it work for the final function (I think): 
#123123 --> dwelling_id
#smart (everywhere) --> replace with df
#6767 --> NaN_count for df


SyntaxError: invalid syntax (<ipython-input-35-969314d31b42>, line 10)

In [30]:
smart.isnull().sum()[0]

2947

In [31]:
#Delete rows with NaN in 'ePower' column
smart_final=smart.dropna(subset=['ePower'])
smart_final.isnull().sum()

eMeter             0
eMeterReturn       0
eMeterLow          0
eMeterLowReturn    0
ePower             0
ePowerReturn       0
dtype: int64

# Working with gas dataframe

In [ ]:
#Inspect for NaNs
gas_nan=gas.isnull().any(axis=1)
print(gas[gas_nan])

In [ ]:
#Interpolate
gas_final = gas.resample('10s').interpolate(method='time')
# Create a ''gasPower'' column. This equals the gas used in that hour
gas_final['gasPower'] = gas_final['gasMeter'].diff()
# First entry is NaN, replace NaN by second entry
gas_final['gasPower'][0] = gas_final['gasPower'][1]
print(gas_final)

In [ ]:
#Fit datetimes
#gas_selected=gas_final.ix[smart_final.index[0]:gas_final.index[-1]]
#print(gas_selected)

In [ ]:
# Gas datetime ends at 23:00:00 (at least in this specific file) and Electricity date time ends at 23:59:50 so
# when merging final DateTime will be 23:00:00
df = pd.merge(smart_final,gas_final,left_index=True, right_index=True)

In [ ]:
print(df)